# Shor's Algorithm on Tket

## Motivation – RSA Cryptosystem 

Bob wants to receive a encoded message from Alice such that only he can read it. What can he do?

1. He picks two very large (200+ bit) prime numbers $p$ and $q$
2. He sends Alice their product $N=pq$ and an encoding constant $c$ through a public channel

**Note:** $c$ must have no factors in common with $\phi(N)$, where $\phi$ is the Euler totient function. Also, $p$ and $q$ seperately must remain private. This is not a hard premise to satisfy, as factoring a ~400-bit number is unfeasible with currently available computers.

## TKET Implementation

In this example we will solve the period finding problem for $a=7$ and $N=15$. We provide the circuits for $U$ where:

$$U|y\rangle = |ay\bmod 15\rangle $$

without explanation. To create $U^x$, we will simply repeat the circuit $x$ times. In the next section we will discuss a general method for creating these circuits efficiently. The function `c_amod15` returns the controlled-U gate for `a`, repeated `power` times.

### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit.visualization import plot_histogram

#Tket port
from pytket import Circuit, OpType
from pytket.circuit import CircBox, QControlBox
from pytket.circuit.display import render_circuit_jupyter

from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction


from pytket.extensions.qiskit import AerBackend
b = AerBackend()                            

print("Imports Successful")

### Definition of the Unitary gate – Multiplication Modulo 15

In [2]:
def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = Circuit(4, name="%i^%i mod 15" % (a, power))     
    for iteration in range(power):
        if a in [2,13]:
            U.SWAP(0,1)
            U.SWAP(1,2)
            U.SWAP(2,3)
        if a in [7,8]:
            U.SWAP(2,3)
            U.SWAP(1,2)
            U.SWAP(0,1)
        if a in [4, 11]:
            U.SWAP(1,3)
            U.SWAP(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.X(q)


    cU = CircBox(U)

    box = QControlBox(cU, 1)
    
    return box

### Defining the Inverse QFT Gate
A `CircBox` is used as to create a 

In [ ]:
def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    qc = Circuit(n, name="QFT†")
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.SWAP(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.add_gate(OpType.CU1, [-1/float(2**(j-m))], [m, j])
        qc.H(j)
    return CircBox(qc)